In [9]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models

# =========================
# Define class names (EuroSAT RGB has 10 classes)
# =========================
class_names = [
    "AnnualCrop",
    "Forest",
    "HerbaceousVegetation",
    "Highway",
    "Industrial",
    "Pasture",
    "PermanentCrop",
    "Residential",
    "River",
    "SeaLake"
]

# =========================
# Load model
# =========================
model = models.resnet18(num_classes=10)
state_dict = torch.load(
    r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\satelite\test 2\eurosat_resnet18.pth",
    map_location="cpu",
    weights_only=True  # safer
)
model.load_state_dict(state_dict)
model.eval()

# =========================
# Define image transform (resize to 64x64, same as EuroSAT)
# =========================
transform = transforms.Compose([
    transforms.Resize((64, 64)),   # or whatever you trained with
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.344, 0.380, 0.408],
                         std=[0.176, 0.176, 0.177])
])


  # adjust if you used different normalization


# =========================
# Load a test image
# =========================
img_path = r"D:\datasets\port_59a30dff58.webp"# change this
image = Image.open(img_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0)  # add batch dimension

# =========================
# Get prediction
# =========================
with torch.no_grad():
    outputs = model(input_tensor)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

# Get top prediction
pred_class_idx = torch.argmax(probabilities).item()
pred_class_name = class_names[pred_class_idx]
confidence = probabilities[pred_class_idx].item() * 100

print(f"Predicted class: {pred_class_name}")
print(f"Confidence: {confidence:.2f}%")

# Show top-3 predictions
top3_probs, top3_idx = torch.topk(probabilities, 3)
print("\nTop 3 Predictions:")
for i in range(3):
    print(f"{class_names[top3_idx[i].item()]}: {top3_probs[i].item()*100:.2f}%")


Predicted class: Residential
Confidence: 59.42%

Top 3 Predictions:
Residential: 59.42%
River: 26.50%
HerbaceousVegetation: 6.57%
